# Lexer

In [187]:
#Parser
import re

class Token:
    def __init__(self, type, value):
        self.type = type
        self.value = value

    def __repr__(self):
        return f"Token({self.type}, {repr(self.value)})"

def lex(source_code):
    # Tokens are defined as pairs of (regex, type)
    token_specification = [
        ('STRING', r'\".*?\"'),        # String
        ('NUMBER',   r'\d+(\.\d*)?'),  # Integer or decimal number
        ('ASSIGN',   r'='),            # Assignment operator
        ('END',      r';'),            # Statement terminator
        ('ID',       r'[A-Za-z_]\w*'), # Identifiers
        ('OPAREN',   r'\('),           # Open parenthesis
        ('CPAREN',   r'\)'),           # Close parenthesis
        ('OBRACE',   r'\{'),           # Open brace
        ('CBRACE',   r'\}'),           # Close brace
        ('COMMA',    r','),            # Comma
        ('PLUS',     r'\+'),           # Plus operator
        ('MINUS',    r'-'),            # Minus operator
        ('MULTIPLY', r'\*'),           # Multiply operator
        ('DIVIDE',   r'/'),            # Divide operator
        ('EQUAL',    r'@'),            # Equal operator
        ('NOTEQUAL', r'!'),            # Not equal operator
        ('LESS',     r'<'),             # Less than operator
        ('GREATER',  r'>'),             # Greater than operator
        ('AND',      r'&'),            # Logical AND
        ('OR',       r'\|\|'),          # Logical OR
        ('NOT',      r'!'),             # Logical NOT
        ('NEWLINE',  r'\n'),           # Line endings
        ('SKIP',     r'[ \t]+'),       # Skip over spaces and tabs
        ('MISMATCH', r'.'),            # Any other character
    ]

    # Keywords and their corresponding types
    keywords = {
        'andika': 'PRINT',
        'kama': 'IF',
        'lasivyo': 'ELSE',
        'vunja': 'BREAK',
        'endelee': 'CONTINUE',
        'kwa': 'FOR',
        'wakati': 'WHILE',
        'kazi': 'FUNCTION',
        'rudisha': 'RETURN',
    }

    # Regex patterns
    token_regex = '|'.join(f'(?P<{pair[0]}>{pair[1]})' for pair in token_specification)
    line_num = 1
    line_start = 0
    tokens = []

    for mo in re.finditer(token_regex, source_code):
        kind = mo.lastgroup
        value = mo.group()
        column = mo.start() - line_start

        if kind == 'NUMBER':
            value = float(value) if '.' in value else int(value)
        elif kind == 'ID' and value in keywords:
            kind = keywords[value]
        elif kind == 'STRING':
            value = value.strip('"')
        elif kind == 'NEWLINE':
            line_start = mo.end()
            line_num += 1
            continue
        elif kind == 'SKIP':
            continue
        elif kind == 'MISMATCH':
            raise RuntimeError(f'{value!r} unexpected on line {line_num}')

        token = Token(kind, value)
        tokens.append(token)

    return tokens

In [188]:
source_code = """
kazi jumuisha(x,y){
  jibu = x+y;
  andika jibu;
}

moja = 1;
mbili = 2;
neno = "word";
andika neno;
jumuisha(moja, mbili);

kama(moja > mbili){
  andika "story za jaba";
}lasivyo{
  andika "all good";
}

"""

tokens = lex(source_code)
for token in tokens:
    print(token)


Token(FUNCTION, 'kazi')
Token(ID, 'jumuisha')
Token(OPAREN, '(')
Token(ID, 'x')
Token(COMMA, ',')
Token(ID, 'y')
Token(CPAREN, ')')
Token(OBRACE, '{')
Token(ID, 'jibu')
Token(ASSIGN, '=')
Token(ID, 'x')
Token(PLUS, '+')
Token(ID, 'y')
Token(END, ';')
Token(PRINT, 'andika')
Token(ID, 'jibu')
Token(END, ';')
Token(CBRACE, '}')
Token(ID, 'moja')
Token(ASSIGN, '=')
Token(NUMBER, 1)
Token(END, ';')
Token(ID, 'mbili')
Token(ASSIGN, '=')
Token(NUMBER, 2)
Token(END, ';')
Token(ID, 'neno')
Token(ASSIGN, '=')
Token(STRING, 'word')
Token(END, ';')
Token(PRINT, 'andika')
Token(ID, 'neno')
Token(END, ';')
Token(ID, 'jumuisha')
Token(OPAREN, '(')
Token(ID, 'moja')
Token(COMMA, ',')
Token(ID, 'mbili')
Token(CPAREN, ')')
Token(END, ';')
Token(IF, 'kama')
Token(OPAREN, '(')
Token(ID, 'moja')
Token(GREATER, '>')
Token(ID, 'mbili')
Token(CPAREN, ')')
Token(OBRACE, '{')
Token(PRINT, 'andika')
Token(STRING, 'story za jaba')
Token(END, ';')
Token(CBRACE, '}')
Token(ELSE, 'lasivyo')
Token(OBRACE, '{')
Token(

# Parser

In [182]:
#Parser
class ASTNode:
    def __init__(self, type, value=None, children=None):
        self.type = type
        self.value = value
        self.children = children if children is not None else []

class ParserError(Exception):
    pass

class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = None
        self.next_token()

    def next_token(self):
        if self.tokens:
            self.current_token = self.tokens.pop(0)
        else:
            self.current_token = None

    def eat(self, token_type):
        if self.current_token is None or self.current_token.type != token_type:
            raise ParserError(f"Expected token {token_type}, got {self.current_token.type if self.current_token else 'None'}")
        self.next_token()

    def parse(self):
        return self.program()

    def program(self):
        nodes = []
        while self.current_token is not None:
            if self.current_token.type == 'FUNCTION':
                nodes.append(self.function_definition())
            else:
                nodes.append(self.statement())
        return ASTNode('program', children=nodes)

    def function_definition(self):
        self.eat('FUNCTION')
        func_name = self.current_token.value
        self.eat('ID')
        self.eat('OPAREN')
        params = self.parameters()
        self.eat('CPAREN')
        self.eat('OBRACE')
        body = self.block()
        self.eat('CBRACE')
        return ASTNode('function_definition', value=func_name, children=[params, body])

    def parameters(self):
        params = []
        while self.current_token and self.current_token.type == 'ID':
            params.append(ASTNode('parameter', value=self.current_token.value))
            self.eat('ID')
            if self.current_token.type == 'COMMA':
                self.eat('COMMA')
        return params

    def block(self):
        statements = []
        while self.current_token and self.current_token.type != 'CBRACE':
            statements.append(self.statement())
        return ASTNode('block', children=statements)

    def statement(self):
        if self.current_token.type == 'IF':
            return self.if_statement()
        elif self.current_token.type == 'FOR':
            return self.for_statement()
        if self.current_token.type == 'ID':
            if self.peek() == 'ASSIGN':
                return self.assignment_statement()
            elif self.peek() == 'OPAREN':
                return self.function_call()
            else:
                raise ParserError(f"Unrecognized statement starting with ID: {self.current_token.value}")
        elif self.current_token.type == 'PRINT':
            return self.print_statement()
        else:
            raise ParserError("Invalid statement")

    def assignment_statement(self):
        var_name = self.current_token.value
        self.eat('ID')
        self.eat('ASSIGN')
        value = self.expression()
        self.eat('END')
        return ASTNode('assignment_statement', value=var_name, children=[value])

    def function_call(self):
        func_name = self.current_token.value
        self.eat('ID')
        self.eat('OPAREN')
        args = self.arguments()
        self.eat('CPAREN')
        self.eat('END')
        return ASTNode('function_call', value=func_name, children=args)

    def arguments(self):
        args = []
        while self.current_token and self.current_token.type != 'CPAREN':
            args.append(self.expression())
            if self.current_token.type == 'COMMA':
                self.eat('COMMA')
        return args

    def print_statement(self):
        self.eat('PRINT')
        expressions = []
        expressions.append(self.expression())
        self.eat('END')
        return ASTNode('print_statement', children=expressions)


    def if_statement(self):
        self.eat('IF')
        self.eat('OPAREN')
        condition = self.comparison()
        self.eat('CPAREN')
        self.eat('OBRACE')
        true_block = self.block()
        self.eat('CBRACE')

        false_block = None
        if self.current_token and self.current_token.type == 'ELSE':
            self.eat('ELSE')
            self.eat('OBRACE')
            false_block = self.block()
            self.eat('CBRACE')

        return ASTNode('if_statement', children=[condition, true_block, false_block])

    def for_statement(self):
        self.eat('FOR')
        self.eat('OPAREN')
        # Optional initialization part
        init = self.assignment_statement() if self.current_token.type == 'ID' else None
        self.eat('END')
        # Condition part
        condition = self.expression()
        self.eat('END')
        # Optional increment part
        increment = self.assignment_statement() if self.current_token.type == 'ID' else None
        self.eat('CPAREN')
        self.eat('OBRACE')
        body = self.block()
        self.eat('CBRACE')

        return ASTNode('for_statement', children=[init, condition, increment, body])


    def expression(self):
      node = self.term()

      # Handle additional terms separated by PLUS
      while self.current_token and self.current_token.type == 'PLUS':
          self.eat('PLUS')
          right = self.term()
          node = ASTNode('add', children=[node, right])

      # Handle additional terms separated by MINUS
      while self.current_token and self.current_token.type == 'MINUS':
          self.eat('MINUS')
          right = self.term()
          node = ASTNode('subtract', children=[node, right])
      # Handle additional terms separated by MULTIPLY
      while self.current_token and self.current_token.type == 'MULTIPLY':
          self.eat('MULTIPLY')
          right = self.term()
          node = ASTNode('multiply', children=[node, right])

      # Handle additional terms separated by DIVIDE
      while self.current_token and self.current_token.type == 'DIVIDE':
          self.eat('DIVIDE')
          right = self.term()
          node = ASTNode('divide', children=[node, right])

      return node

    def comparison(self):
      node = self.term()

      # handle comparison operators like LESS, GREATER, LESSEQUAL, GREATEREQUAL, EQUAL, NOTEQUAL, etc.
      # Example for handling one comparison operator:
      while self.current_token and self.current_token.type == 'LESS':
          self.eat('LESS')
          right = self.term()
          node = ASTNode('less_than', children=[node, right])
      while self.current_token and self.current_token.type == 'GREATER':
          self.eat('GREATER')
          right = self.term()
          node = ASTNode('greater_than', children=[node, right])
      while self.current_token and self.current_token.type == 'EQUAL':
          self.eat('EQUAL')
          right = self.term()
          node = ASTNode('equal', children=[node, right])
      while self.current_token and self.current_token.type == 'NOTEQUAL':
          self.eat('NOTEQUAL')
          right = self.term()
          node = ASTNode('not_equal', children=[node, right])

      return node

    def term(self):
        # A term is either a number, a string, or an identifier
        token = self.current_token
        if token.type == 'NUMBER':
            self.eat('NUMBER')
            return ASTNode('number', value=token.value)
        elif token.type == 'STRING':
            self.eat('STRING')
            return ASTNode('string', value=token.value)
        elif token.type == 'ID':
            self.eat('ID')
            return ASTNode('identifier', value=token.value)
        else:
            raise ParserError(f"Invalid term: Unexpected token {token.type} with value '{token.value}'")
    def peek(self):
        return self.tokens[0].type if self.tokens else None



# Example usage:
tokens = lex(source_code)
parser = Parser(tokens)
ast = parser.parse()


# Interpreter

In [183]:
#Interpreter
class InterpreterError(Exception):
    pass

class Interpreter:
    def __init__(self):
        self.context = {}

    def interpret(self, node):
        method_name = 'interpret_' + node.type
        method = getattr(self, method_name, self.no_visit_method)
        return method(node)

    def no_visit_method(self, node):
        raise InterpreterError(f"No interpret_{node.type} method")

    def interpret_program(self, node):
        result = None
        for child in node.children:
            result = self.interpret(child)
        return result

    def interpret_function_definition(self, node):
        self.context[node.value] = node
        # Assuming functions don't return a value upon definition
        return None

    def interpret_function_call(self, node):
        function_node = self.context.get(node.value)
        if not function_node:
            raise InterpreterError(f"Function {node.value} not defined")

        if len(function_node.children[0]) != len(node.children):
            raise InterpreterError(f"Function {node.value} takes {len(function_node.children[0])} arguments but {len(node.children)} were given")

        # Save the current context
        saved_context = self.context.copy()

        # Update the context with arguments
        for param, arg in zip(function_node.children[0], node.children):
            self.context[param.value] = self.interpret(arg)

        # Interpret the function body
        result = self.interpret(function_node.children[1])

        # Restore the original context
        self.context = saved_context

        return result

    def interpret_block(self, node):
        result = None
        for statement in node.children:
            result = self.interpret(statement)
        return result

    def interpret_print_statement(self, node):
        value = self.interpret(node.children[0])
        print(value)
        return value

    def interpret_assignment_statement(self, node):
        var_name = node.value
        value = self.interpret(node.children[0])
        self.context[var_name] = value
        return value

    def interpret_identifier(self, node):
        var_name = node.value
        if var_name not in self.context:
            raise InterpreterError(f"Undefined variable '{var_name}'")
        return self.context[var_name]

    def interpret_number(self, node):
        return node.value

    def interpret_parameter(self, node):
        # Parameters are handled during function calls, but if this method is called,
        # it suggests a parameter is being used as a standalone expression, which is an error.
        raise InterpreterError(f"Parameter {node.value} cannot be used as an expression")

    # Add methods for handling expressions, like addition, subtraction, etc.
    # Assuming you have node types 'add', 'subtract', 'multiply', 'divide'

    def interpret_string(self, node):
        return node.value.strip('"')

    def interpret_add(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val + right_val

    def interpret_subtract(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val - right_val

    def interpret_multiply(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val * right_val

    def interpret_divide(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        if right_val == 0:
            raise InterpreterError("Division by zero")
        return left_val / right_val


    def interpret_if_statement(self, node):
      condition = self.interpret(node.children[0])
      if condition:
          return self.interpret(node.children[1])  # true_block
      elif node.children[2] is not None:
          return self.interpret(node.children[2])  # false_block


    def interpret_less_than(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val < right_val

    def interpret_greater_than(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val > right_val

    def interpret_less_equal(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val <= right_val

    def interpret_greater_equal(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val >= right_val

    def interpret_equal(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val == right_val

    def interpret_not_equal(self, node):
        left_val = self.interpret(node.children[0])
        right_val = self.interpret(node.children[1])
        return left_val != right_val

    # ...and so on for other operations and language constructs.


# Assume `ast` is the abstract syntax tree returned by the parser
interpreter = Interpreter()
interpreter.interpret(ast)


word
3
all good


'all good'

# Example Usage

In [194]:
#Example Usage

#@ is used for "=="

source_code = """
kazi jumuisha(x,y){
  jibu = x+y;
  andika jibu;
}

moja = 3;
mbili = 2;

neno = "Hello world";
andika neno;

jumuisha(moja, mbili);

kama(moja > mbili){
  andika "story za jaba";
}lasivyo{
  andika "all good";
}

"""

# Step 1: Tokenize the source code
tokens = lex(source_code)

# Step 2: Parse the tokens into an AST
parser = Parser(tokens)
try:
    ast = parser.parse()
except ParserError as e:
    print(f"Parser error: {e}")
    ast = None

# Step 3: Interpret the AST
if ast:
    interpreter = Interpreter()
    try:
        result = interpreter.interpret(ast)
        # If you want to return something after the whole program has run,
        # you can use the `result` variable which holds the last evaluated expression.
        print(" ")
        print("Compiled Successfully.")
    except InterpreterError as e:
        print(f"Interpreter error: {e}")


Hello world
5
story za jaba
 
Compiled Successfully.
